# Instructions



1.   Import the excel file with the song data by clicking the file icon on the left and dragging the file over the sidebar.
2.   Leave the "Setup" and "Functions" sections untoggled, and click the play button inside the brackets to the left of "# cells hidden" for each section.
3. If the "Code" section is untoggled, toggle it so you can see each block of code.
4. From top to bottom, click the play button on the top left of each cell
5. Cell is done running when green check shows up to the left of the cell (keep in minds, some cells will take minutes to run because they are retrieving data over the internet).
6. Enter input when required
7. Output file will show up on the sidebar on the left



# Methodology

To get the song data needed for the search (i.e., the song title and the artist), I used the Spotify link. Consequently, if a song didn't have a Spotify link, I didn't search for its charting history.

For the songs that I could find, I used Music Charts Archive to pull the data because it was the only website I could find that could provide the start date of every week a song was on the Hot 100. The Music Charts Archive page for Drunk in Love by Beyonce is [here](https://musicchartsarchive.com/singles/beyonce/drunk-in-love) so you can see what I'm referring to. For every song I was able to find on this website, I checked to see if the last week the song charted occurred before the date the survey was started. If this was the case, I just used the "Peak Position" and "Number of Weeks on Chart" information that was present on the page. If this was not the case, I calculated the number of weeks and peak position using the information in the table.

I noticed that Music Charts Archive doesn't have a dedicated page for every song that has ever charted. Because of this, if I couldn't find the song on the website, I double checked for the song on the artist's Billboard chart history page (example [here](https://www.billboard.com/artist/waka-flocka-flame/chart-history/hsi/)). Unfortunately, the only information on this page is the peak position of the song, the week it debuted, the week it peaked, and the number of weeks it was on the chart. This means I can't be 100% certain that the song completed its chart run before the survey was started. I assumed that the songs charted consecutively starting from the week it debuted, which I know is not always the case, so I added 5 weeks as a margin of error.

If there's a song with a spotify link but no url, there was neither a Music Charts Archive page nor a Billboard page for the artist

# Setup

In [ ]:
import requests
import re
import pandas as pd
import difflib

In [ ]:
def get_authorization_headers():
  AUTH_URL = 'https://accounts.spotify.com/api/token'
  CLIENT_ID = '*'
  CLIENT_SECRET = '*'

  # POST
  auth_response = requests.post(AUTH_URL, {
      'grant_type': 'client_credentials',
      'client_id': CLIENT_ID,
      'client_secret': CLIENT_SECRET,
  })

  # convert the response to JSON
  auth_response_data = auth_response.json()

  # save the access token
  access_token = auth_response_data['access_token']

  headers = {
      'Authorization': 'Bearer {token}'.format(token=access_token)
  }

  return headers

In [ ]:
global tracked

# Functions

In [ ]:
def get_title_start_and_end_index(html_text):
  start = html_text.find("<li class=\"lrv-u-width-100p")
  end = html_text[start:].find('</li>')
  return (start, end)

In [ ]:
def get_stat_start_and_end_index(html_text):
  start = html_text.find("<ul class=\"lrv-a-unstyle-list")
  end = html_text[start:].find('</ul>')
  return (start, end)

In [ ]:
def sanitize_html(html_text):
  text = re.sub('<[^<]+?>', '', html_text)
  return text

In [ ]:
def remove_tabs(text):
  text = text.replace("\t", "")
  return text

In [ ]:
def process_text(text):
  text = text.replace("&#039;", "'")
  text = text.replace("&amp;", "&")
  return text

In [ ]:
def extract_title_and_artists(text):
  text_list = text.splitlines()
  res = []
  for t in text_list:
    if len(t) > 0:
      res.append(t)
  return res

In [ ]:
def extract_stats(text):
  text_list = text.splitlines()
  res = []
  for t in text_list:
    if len(t) > 0 and len(t) < 4:
      res.append(t)
    if len(res) == 3:
      return res
  return res

In [ ]:
def get_raw(text):
  return re.sub(r'[^a-zA-Z]+', '', text).lower()

In [ ]:
def split_spotify_date(date):
  year, month, date = date.split("-")
  return [year, month, date]

In [ ]:
def get_chart_from_date(spotify_date):
  year, month, day = split_spotify_date(spotify_date)
  response = requests.get(f"https://www.billboard.com/charts/hot-100/{year}-{month}-{day}")

  chart_dict = {"title": [],
                "raw_title": [],
                "artists": [],
                "raw_artists": [],
                "current_pos": [],
                "peak_pos": [],
                "wks_on_chart": []}

  html_text = response.text
  current_pos = 1
  for i in range(200):
    start, end = get_title_start_and_end_index(html_text)
    raw_text= html_text[start:start+end]
    text = sanitize_html(raw_text)
    text = remove_tabs(text)
    text = process_text(text)

    stat_start, stat_end = get_stat_start_and_end_index(html_text)
    raw_stats = html_text[stat_start:stat_start+stat_end]
    stats = sanitize_html(raw_stats)
    stats = remove_tabs(stats)

    if len(text.strip("\n")) != 0:
      title, artists = extract_title_and_artists(text)
      _, peak_pos, wks_on_chart = extract_stats(stats)
      chart_dict["title"].append(title)
      chart_dict["raw_title"].append(get_raw(title))
      chart_dict["artists"].append(artists)
      chart_dict["raw_artists"].append(get_raw(artists))
      chart_dict["current_pos"].append(current_pos)
      chart_dict["peak_pos"].append(peak_pos)
      chart_dict["wks_on_chart"].append(wks_on_chart)
      current_pos += 1

    html_text = html_text[start+end:]

  chart = pd.DataFrame(chart_dict)
  return chart

In [ ]:
def get_spotify_track(spotify_id):
  spotify_date = requests.get(f"https://api.spotify.com/v1/tracks/{spotify_id}", headers=SPOTIFY_HEADERS).json()
  return spotify_date

In [ ]:
def get_track_artist(track_data):
  try:
    artist = track_data['artists'][0]['name']
    artist = artist.replace(" ", "-")
    artist = artist.replace(".", "")
  except:
    print(track_data)

  return artist

In [ ]:
def get_track_name(track_data):
  name = track_data['name']

  feat_in_name = name.find("(")
  if feat_in_name > 0:
    name = name[:feat_in_name].strip()

  name = name.replace(" ", "-")
  return re.sub(r'[^a-zA-Z0-9\-\?\!\&]+', '', name).lower()

In [ ]:
def get_track_id_from_hyperlink(hyperlink):
  hyperlink = hyperlink.replace(" ", "")
  i = hyperlink.find("track/") + 6
  return hyperlink[i:i+22]

In [ ]:
def get_track_artist_and_name(survey_data):
  spotify_hyperlink = survey_data["hyperlink to song"]
  if not spotify_hyperlink or spotify_hyperlink[0:4] != "http":
    return None

  spotify_track_id = get_track_id_from_hyperlink(spotify_hyperlink)
  track_data = get_spotify_track(spotify_track_id)
  artist = get_track_artist(track_data)
  track_name = get_track_name(track_data)
  return [artist, track_name]

In [ ]:
def extract_number(html_text):
  return re.sub(r'[^0-9]+', '', html_text)

In [ ]:
from datetime import datetime

def get_relevant_chart_info(title, artist):
  url = f"https://musicchartsarchive.com/singles/{artist}/{title}"
  response = requests.get(url, headers={'Accept': '*/*', 'User-Agent': 'request',})

  if response.status_code != 200:
    return [-1 for i in range(5)]

  text = response.text
  peak_ind = text.find("Peak Position:")
  num_week_ind = text.find("Number of Weeks on Chart:")
  if peak_ind == -1:
    return [0 for i in range(5)]

  peak_pos_dirty = text[peak_ind: peak_ind+25]
  num_weeks_dirty = text[num_week_ind:num_week_ind+35]
  peak_pos = extract_number(peak_pos_dirty)
  num_weeks = extract_number(num_weeks_dirty)

  last_table_entry_ind = text.find("</table>")
  date = re.sub("[^0-9]+", "", text[last_table_entry_ind-100: last_table_entry_ind])[:8]
  date = datetime.strptime(date, "%Y%m%d").date()
  return [int(peak_pos), int(num_weeks), date, url, ""]

In [ ]:
def manually_calculate_weeks_and_peak_pos(url, survey_date):
  res = requests.get(url, headers={'Accept': '*/*', 'User-Agent': 'request',})
  r = res.text
  peak_pos = 101
  weeks = 0
  while True:
    s = "/singles-chart/"
    i = r.find(s) + len(s)
    date = r[i:i+10]
    if date.count('-') != 2:
      break
    date = re.sub("[^0-9]+", "", date)
    date = datetime.strptime(date, "%Y%m%d").date()
    pos = re.sub("[^0-9]+", "", r[i+49:i+52])
    if (date <= survey_date):
      peak_pos = min(peak_pos, int(pos))
      weeks += 1
    else:
      break

    r = r[i+10:]

  return [weeks, peak_pos if weeks != 0 else 0]

In [ ]:
def get_peak_and_weeks_and_url_from_music_archive(survey_data):
  if not survey_data["track_data_for_search"]:
    return [-1 for _ in range(4)]
  artist, title = survey_data["track_data_for_search"]
  if (artist, title) not in tracked:
    tracked[(artist, title)] = get_relevant_chart_info(title, artist)

  peak_pos, num_weeks, last_date, url, _ = tracked[(artist, title)]
  if peak_pos > 0:
    survey_date = survey_data["Survey_Started_Date"].to_pydatetime().date()
    if survey_date < last_date:
      print("Manual calculation for", title)
      num_weeks, peak_pos = manually_calculate_weeks_and_peak_pos(url, survey_date)
      return [peak_pos, num_weeks, url, ""]

  return [peak_pos, num_weeks, url, ""]

In [ ]:
def get_peak_pos(survey_data):
  return int(survey_data["final_stats"][0])

In [ ]:
def get_num_weeks(survey_data):
  return int(survey_data["final_stats"][1])

In [ ]:
def get_title_from_history(text):
  text = re.sub(r'[^a-zA-Z0-9\'\?\!\ \*\/\&]+', '', text)
  end = text.find("h3span")
  return text[:end].strip("/")

In [ ]:
def get_num_from_text(text):
  text = re.sub(r'[^0-9\ ]+', '', text)
  return text.strip(" ").split(" ")[0]

In [ ]:
def get_peak_from_history(text):
  peak_string = 'aria-label="Peak'
  j = text.find(peak_string) + len(peak_string)
  proc_text = process_text(text[j:j+20])
  peak = get_num_from_text(proc_text)
  return peak

In [ ]:
def get_week_from_history(text):
    week_string = 'aria-label="Weeks on Chart '
    k = text.find(week_string) + len(week_string)
    proc_text = process_text(text[k:k+30])
    week = get_num_from_text(proc_text)
    return week

In [ ]:
def get_date_from_history(text):
  date_string = 'href="https://www.billboard.com/charts/hot-100/'
  i = text.find(date_string) + len(date_string) + 36
  proc_text = process_text(text[i:i+26])
  text = re.sub(r'[^0-9\.]+', '', proc_text).strip()
  if len(text) == 0:
    return None
  month, day, year = text.split(".")
  if int(year) > 23:
    year = "19" + year
  else:
    year = "20" + year

  date = datetime.strptime("".join([year, month, day]), "%Y%m%d").date()
  return date

In [ ]:
def address_special_cases(artist):
  if artist.lower() == "p!nk":
    return "pink"
  artist = artist.replace("’", "")
  return artist

In [ ]:
def get_track_name_df(df):
  name = df['title']

  feat_in_name = name.find("(")
  if feat_in_name > 0:
    name = name[:feat_in_name].strip()

  name = name.replace(" ", "-")
  return re.sub(r'[^a-zA-Z0-9\-\?\!\&]+', '', name).lower()

In [ ]:
def get_history_dict(text):
  history_chart = {"title": [], "peak": [], "weeks": [], "date": []}
  title = ""
  while True:
    title_string = '<h3 id="title-of-a-story" class="c-title  a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only artist-chart-row-title">'
    i = text.find(title_string) + len(title_string)
    proc_text = process_text(text[i:i+200])
    title = get_title_from_history(proc_text)
    text = text[i:]
    if title.find("artrow") != -1 or i == 281:
      break

    peak = get_peak_from_history(text)
    week = get_week_from_history(text)
    date = get_date_from_history(text)
    history_chart["title"].append(title)
    history_chart["peak"].append(peak)
    history_chart["weeks"].append(week)
    history_chart["date"].append(date)

  return history_chart

In [ ]:
from requests.sessions import TooManyRedirects
def get_billboard_page_and_url(artist):
  billboard_history = ""
  used_url = url = f"https://www.billboard.com/artist/{artist}/chart-history/hsi/"
  second_try_url = f"https://www.billboard.com/artist/{artist+'-2'}/chart-history/hsi/"
  encountered_error = False
  try:
    billboard_history = requests.get(url)
  except TooManyRedirects:
    print(f"Too many redirects on {url}")
    encountered_error = True

  table_empty = (not billboard_history or billboard_history.text.find("artist-chart-history-container") == -1)
  if not billboard_history or billboard_history.status_code != 200 or encountered_error or table_empty:
    try:
      billboard_history = requests.get(second_try_url)
      if billboard_history.status_code == 200:
        used_url = second_try_url
        print(f"Found {artist} on second try")
      else:
        return [None, None]
    except TooManyRedirects:
      print(f"Too many redirects on {second_try_url}")
      return [None, None]

  text = billboard_history.text
  if (text.find('artist-chart-history-container') == -1):
    billboard_history = requests.get(second_try_url)
    text = billboard_history.text

  return [text, used_url]

In [ ]:
def search_billboard_history(artist, search_title):
  close_match = None
  artist = address_special_cases(artist)
  billboard_history, url = get_billboard_page_and_url(artist)

  error = [-1 for _ in range(5)]
  found_artist_but_not_song = [-1, -1, -1, url, close_match]
  if not billboard_history:
    return error

  history_dict = get_history_dict(billboard_history)
  if len(history_dict["title"]) == 0:
    return found_artist_but_not_song

  df = pd.DataFrame(history_dict)
  df["title"] = df.apply(get_track_name_df, axis=1)
  row = df.loc[df.title == search_title]
  if row.shape[0] == 0:
    close_match_list = difflib.get_close_matches(search_title, df.title.tolist())
    if len(close_match_list):
      close_match = close_match_list[0]
      print(f"Found close match! Actual: {search_title}, Found: {close_match}")
      row = df.loc[df.title == close_match]
    else:
      return found_artist_but_not_song

  row = row.iloc[0]
  return [row["peak"], row["weeks"], row["date"], url, close_match]

In [ ]:
from datetime import timedelta

def double_check_billboard(df):
  if not df["track_data_for_search"] or len(df["track_data_for_search"]) < 2:
    return df["final_stats"]
  if df["final_stats"][0] != -1:
    return df["final_stats"]

  artist, title = df["track_data_for_search"]
  if title.find("---") != -1:
    title = title[:title.find("---")]

  if (artist, title) in tracked:
    return tracked[(artist, title)]

  tracked[(artist, title)] = search_billboard_history(artist, title)
  weeks = tracked[(artist, title)][1]
  date = tracked[(artist, title)][2]
  if not isinstance(date, int) and date + timedelta(days=(7*(int(weeks) + 5))) > df["Survey_Started_Date"].to_pydatetime().date():
    return [999, 999] + tracked[(artist, title)][-2:]

  return tracked[(artist, title)][:2] + tracked[(artist, title)][-2:]

In [ ]:
def get_url(df):
  if df["final_stats"][-2] == -1:
    return ""

  return df["final_stats"][-2]

In [ ]:
def get_close_match(df):
  return df["final_stats"][-1]

In [ ]:
from os import listdir
from os.path import isfile, join
def get_excel_file():
  mypath = "/content"
  files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
  for f in files:
    if f.find('.xlsx') != -1:
      print("Found excel file:", f)
      return f

# Code

In [ ]:
file = get_excel_file()

Found excel file: TEAM300_music.xlsx


In [ ]:
survey_data = pd.read_excel(file)
survey_data.fillna("", inplace=True)

In [ ]:
SPOTIFY_HEADERS = get_authorization_headers()

In [ ]:
survey_data["track_data_for_search"] = survey_data.apply(get_track_artist_and_name, axis=1)

In [ ]:
survey_data["final_stats"] = survey_data.apply(get_peak_and_weeks_and_url_from_music_archive, axis=1)

Manual calculation for wishful-drinking
Manual calculation for moscow-mule


In [ ]:
survey_data["final_stats"] = survey_data.apply(double_check_billboard, axis=1)

Found close match! Actual: velvet-heartbreak, Found: heartbeat
Found close match! Actual: in-vein, Found: on-everything
Found close match! Actual: swimming-pools, Found: swimming-pools-drank
Found Panic!-At-The-Disco on second try
Found close match! Actual: one-of-the-drunks, Found: into-the-unknown
Found close match! Actual: seorita, Found: senorita
Found Pierre-Bourne on second try


In [ ]:
survey_data["peak_position"] = survey_data.apply(get_peak_pos, axis=1)
survey_data["num_weeks"] = survey_data.apply(get_num_weeks, axis=1)
survey_data["url"] = survey_data.apply(get_url, axis=1)
survey_data["used_close_match"] = survey_data.apply(get_close_match, axis=1)

In [ ]:
close_matches_found = {}
def verify_data(survey_data):
  actual = survey_data['song title']
  close_match = survey_data['used_close_match']
  print(f"Acutal: {actual}, Close match: {close_match}")
  if close_match in close_matches_found:
    response = close_matches_found[close_match]
  else:
    response = input("Was the right song found? (y/n): ")
    close_matches_found[close_match] = response

  if response == "n":
    return [-1, -1]
  else:
    return [survey_data['peak_position'], survey_data['num_weeks']]

MANUAL INPUT REQUIRED FOR NEXT CELL

You must verify the song found with a similar name is the correct song. If the titles do not refer to the same songs, type "n" (no quotation marks) and press enter. If they refer to the same song, type "y" (no quotation marks) and press enter. For example, "Acutal: Swimming Pool, Close match: swimming-pools-drank" refers to the same song, so you would type "y". On the other hand, "Acutal: "In Vein", Close match: on-everything" clearly do not refer to the same song, so you would type "n".

In [ ]:
used_close_match_df = survey_data.loc[(survey_data.used_close_match.isin([None, '', -1]) == False)]
used_close_match_df['verify_data'] = used_close_match_df.apply(verify_data, axis=1)

Acutal: Velvet Heartbreak, Close match: heartbeat
Was the right song found? (y/n): n
Acutal: "In Vein", Close match: on-everything
Was the right song found? (y/n): n
Acutal: Swimming Pool, Close match: swimming-pools-drank
Was the right song found? (y/n): y
Acutal: One of the Drunks, Close match: into-the-unknown
Was the right song found? (y/n): n
Acutal: Senorita, Close match: senorita
Was the right song found? (y/n): y
Acutal: Swimming Pool, Close match: swimming-pools-drank
Acutal: One of the Drunks, Close match: into-the-unknown
Acutal: Swimming Pool, Close match: swimming-pools-drank
Acutal: Swimming Pools , Close match: swimming-pools-drank
Acutal: Swimming Pools , Close match: swimming-pools-drank


<ipython-input-134-0a621218931d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_close_match_df['verify_data'] = used_close_match_df.apply(verify_data, axis=1)


In [ ]:
for i in used_close_match_df.index.to_list():
  peak_pos, weeks = used_close_match_df.loc[i]["verify_data"]
  survey_data.loc[i, "peak_position"] = peak_pos
  survey_data.loc[i, "num_weeks"] = weeks

In [ ]:
concise_song_data = survey_data[["PID", "Survey_Started_Date", "song_title_artist (what the participant wrote)", "song title", "artist",
                                 "hyperlink to song", "peak_position", "num_weeks", "url"]]

In [ ]:
concise_song_data["Survey_Started_Date"] = concise_song_data["Survey_Started_Date"].dt.date

<ipython-input-137-71d67bfd4b59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concise_song_data["Survey_Started_Date"] = concise_song_data["Survey_Started_Date"].dt.date


In [ ]:
concise_song_data.to_excel("concise_song_data.xlsx", index=False)